# Imports

In [93]:
import  tensorflow                  as      tf
import  numpy                       as      np
import  os
from    sklearn.model_selection     import  train_test_split
from    tensorflow.keras.utils      import  to_categorical

from    tensorflow.keras.models     import  Sequential
from    tensorflow.keras.layers     import  LSTM, Dense
from    tensorflow.keras.callbacks  import  TensorBoard

## Constants

In [94]:
SAMPLES_PER_WORD = 60
FRAMES_PER_SAMPLE = 30

SAMPLES_PATH = "/Volumes/SSD/collection/"

In [97]:
#words = np.array(['hello', 'my', 'none', 'person'])
words = np.array(['none', 'person', 'hello'])
words_path = {
    'hello' : 59,
    'none' : 61,
    'person' : 63
    }

labels_map = {label:num for num, label in enumerate(words)}
labels_map

{'none': 0, 'person': 1, 'hello': 2}

In [98]:

samples, labels = [], []
for word in words:
    for sample in range(SAMPLES_PER_WORD):
        sample_data = []
        for frame in range(FRAMES_PER_SAMPLE):
            path        = os.path.join(SAMPLES_PATH, f'{words_path[word]}', f'{words_path[word]}_{sample}_{frame}.npy')
            frame_data  = np.load(path)
            sample_data.append(frame_data)
        samples.append(sample_data)
        labels.append(labels_map[word])

In [99]:
X = np.array(samples)
X.shape

(180, 30, 225)

In [100]:
y = to_categorical(labels).astype(int)

In [101]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

# Model

In [89]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

model           = Sequential()

hands   = 63 + 63
pose    = 99

lstm_model_1    = LSTM(     64,     return_sequences = True,    activation = "relu", input_shape=(30, hands+pose) )
lstm_model_2    = LSTM(     128,    return_sequences = True,    activation = "relu" )
lstm_model_3    = LSTM(     64,     return_sequences = False,   activation = "relu" )

dense_model_1   = Dense(    64,                                 activation = "relu" )
dense_model_2   = Dense(    32,                                 activation = "relu" )
dense_model_3   = Dense(    words.shape[0],                     activation = "softmax" )

model.add( lstm_model_1 )
model.add( lstm_model_2 )
model.add( lstm_model_3 )

model.add( dense_model_1 )
model.add( dense_model_2 )
model.add( dense_model_3 )

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [90]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [91]:
model.fit(X_train, y_train, epochs=1000)

Epoch 1/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - categorical_accuracy: 0.5066 - loss: 0.6954
Epoch 2/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - categorical_accuracy: 0.4684 - loss: 2.1060
Epoch 3/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - categorical_accuracy: 0.6173 - loss: 0.6384
Epoch 4/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - categorical_accuracy: 0.9060 - loss: 0.7692
Epoch 5/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - categorical_accuracy: 0.6184 - loss: 1.6780
Epoch 6/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - categorical_accuracy: 0.9289 - loss: 0.3726
Epoch 7/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - categorical_accuracy: 0.9243 - loss: 0.1725
Epoch 8/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - categorical_accuracy: 0.8934 - loss: 0.2186
Epoch 9/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - categorical_accuracy: 1.0000 - loss: 0.0324
Epoch 10/1000
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - categorical_accuracy: 1.0000 - loss: 0.0229
Epoch 11/1000
4/4 ━━━━━━━━━━━

In [92]:
model.save('none-person.keras')